# Visualização da Informação

In [2]:
import pandas as pd
import matplotlib as plt

In [3]:
dataFileName = "USD_BRL_hist.csv"

In [8]:
df = pd.read_csv(dataFileName)
df.head()

,Data,USD_BRL
0,31.12.2019,4.0195
1,30.12.2019,4.0195
2,27.12.2019,4.0460
3,26.12.2019,4.0560
4,25.12.2019,4.0838


In [9]:
df["Data"] = pd.to_datetime(df["Data"], dayfirst=True)
df.head()

,Data,USD_BRL
0,2019-12-31,4.0195
1,2019-12-30,4.0195
2,2019-12-27,4.0460
3,2019-12-26,4.0560
4,2019-12-25,4.0838


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2608 entries, 0 to 2607
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Data     2608 non-null   datetime64[ns]
 1   USD_BRL  2608 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.9 KB
